# Custom Training with YOLOv5

We assemble a dataset and train a custom YOLOv5 model to recognize the objects in our dataset. To do so we will take the following steps:

* Gather a dataset of images and label our dataset
* Export our dataset to YOLOv5
* Train YOLOv5 to recognize the objects in our dataset
* Evaluate our YOLOv5 model's performance
* Run test inference to view our model at work



![](https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png)

# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Step 2: Assemble Our Dataset

In order to train our custom model, we need to assemble a dataset of representative images with bounding box annotations around the objects that we want to detect. And we need our dataset to be in YOLOv5 format.

In Roboflow, you can choose between two paths:

* Convert an existing dataset to YOLOv5 format. Roboflow supports over [30 formats object detection formats](https://roboflow.com/formats) for conversion.
* Upload raw images and annotate them in Roboflow with [Roboflow Annotate](https://docs.roboflow.com/annotate).

# Annotate

![](https://roboflow-darknet.s3.us-east-2.amazonaws.com/roboflow-annotate.gif)

# Version

![](https://roboflow-darknet.s3.us-east-2.amazonaws.com/robolfow-preprocessing.png)


##Setting Up Environment

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"


##Cloning Dataset From Github Repository

In [ ]:

# contains modified versions of detect.py, data.yaml
# also contains best.pt !!!
!git clone https://github.com/Ojus999/YOLOv5-ObjectDetection

In [ ]:
# setting path to the newly cloned repo
PATH = "/content/yolov5/YOLOv5-ObjectDetection"

# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
# img arg was 416
# not required as best.pt is there in cloned repo
# !python train.py --img 472 --batch 16 --epochs 50 --data {PATH}/data.yaml --weights yolov5s.pt --cache

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# killing to prevent hanging
!kill 3396

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
# not possible  as we have not trained
# %reload_ext tensorboard
# %tensorboard --logdir runs

##Creating Folder for Cropped Images


In [ ]:
if not os.path.exists('/content/yolov5/Cropped_Imgs'):
  !mkdir {'/content/yolov5/Cropped_Imgs'}

if not os.path.exists('/content/yolov5/Cropped_Imgs/License'):
  !mkdir {'/content/yolov5/Cropped_Imgs/License'}

if not os.path.exists('/content/yolov5/Cropped_Imgs/Face'):
  !mkdir {'/content/yolov5/Cropped_Imgs/Face'}

##Deleting Files in Cropped Images

In [ ]:
if os.path.exists('/content/yolov5/Cropped_Imgs/Face'):
  Path = '/content/yolov5/Cropped_Imgs/Face'
  for file in os.listdir(Path):
    os.unlink(os.path.join(Path,file))

if os.path.exists('/content/yolov5/Cropped_Imgs/License'):
  Path = '/content/yolov5/Cropped_Imgs/License'
  for file in os.listdir(Path):
    os.unlink(os.path.join(Path,file))

##Move Modified Detect.py To The Required Folder

In [ ]:
import shutil


src_path = "/content/yolov5/YOLOv5-ObjectDetection/detect-1.py"
dst_path = "/content/yolov5/detect-1.py"
shutil.move(src_path, dst_path)

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!python detect-1.py --weights {PATH}/best.pt --img 416 --conf 0.7 --source {PATH}/test/images

In [ ]:
#!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.7 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

##Downloading The Cropped Images Folders

In [ ]:
if not os.path.exists('/content/Zip_Files'):
  !mkdir {'/content/Zip_Files'}

In [ ]:
!zip -r "/content/Zip_Files/Face.zip" "/content/yolov5/Cropped_Imgs/Face"
!zip -r "/content/Zip_Files/License.zip" "/content/yolov5/Cropped_Imgs/License"


##Video


In [ ]:
# for displaying og video
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(3,3))  #Display size specification

    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=50, repeat_delay=1000)

    plt.close()
    return anime

In [ ]:
video = imageio.mimread('/content/yolov5/runs/detect/exp4/video2.mp4',memtest=False)  #Loading video
#video = [resize(frame, (256, 256))[..., :3] for frame in video]    #Size adjustment (if necessary)
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5